<a href="https://colab.research.google.com/github/programminghunk/sugarcane/blob/main/experiment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 34960, done.
remote: Counting objects: 100% (3537/3537), done.
remote: Compressing objects: 100% (1527/1527), done.
remote: Total 34960 (delta 2401), reused 3014 (delta 1996), pack-reused 31423
Receiving objects: 100% (34960/34960), 29.57 MiB | 31.64 MiB/s, done.
Resolving deltas: 100% (25160/25160), done.


In [ ]:
!nvdia-smi

/bin/bash: nvdia-smi: command not found


In [ ]:
#%cd ..
#!pip install -e '.[dev]'

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.72 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5771.8/8062.4 GB disk)


In [ ]:
%%writefile ultralytics/cfg/models/v8/backbonev9.yaml

# Parameters
nc: 2 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768] # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512] # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512] # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [32, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [64, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [64, True]]
  - [-1, 1, Conv, [128, 3, 2]] # 3-P3/8
  - [-1, 6, C2f, [128, True]]
  - [-1, 1, SCDown, [256, 3, 2]] # 5-P4/16
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, SCDown, [512, 3, 2]] # 7-P5/32
  - [-1, 3, C2fCIB, [512, True, True]]
  - [-1, 1, SPPF, [512, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)


Overwriting ultralytics/cfg/models/v8/backbonev9.yaml


In [ ]:
%pwd

'/kaggle/working/ultralytics'

In [ ]:
import ultralytics
from ultralytics import YOLO

model = YOLO("/kaggle/working/ultralytics/ultralytics/cfg/models/v8/backbonev9.yaml")

# Display model information (optional)
model.info()

WARNING ⚠️ no model scale passed. Assuming scale='n'.
backbonev9 summary: 252 layers, 1,871,038 parameters, 1,871,022 gradients, 5.6 GFLOPs


(252, 1871038, 1871022, 5.5895552)

In [ ]:
%cd ..

/kaggle/working


In [ ]:
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("np_mynp")
version = project.version(2)
dataset = version.download("yolov8")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.72, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to np_mynp-2 in yolov8:: 100%|██████████| 6719/6719 [00:01<00:00, 6181.28it/s]


In [ ]:
%%writefile np_mynp-2/data.yaml
path: /kaggle/working
names:
- FIT
- UNFIT
nc: 2
roboflow:
  license: CC BY 4.0
  project: np_mynp
  url: https://universe.roboflow.com/buddetection/np_mynp/dataset/1
  version: 2
  workspace: buddetection
test: np_mynp-2/test/images
train: np_mynp-2/train/images
val: np_mynp-2/test/images


Overwriting np_mynp-2/data.yaml


In [ ]:
%%writefile np_mynp-2/data.yaml
path: kaggle/working
names:
- FIT
- UNFIT
nc: 2
roboflow:
  license: CC BY 4.0
  project: np_mynp
  url: https://universe.roboflow.com/buddetection/np_mynp/dataset/2
  version: 2
  workspace: buddetection
test: np_mynp-2/test/images
train: np_mynp-2/train/images
val: np_mynp-2/test/images

Overwriting np_mynp-2/data.yaml


In [ ]:
%pwd

'/kaggle/working'

In [ ]:
results = model.train(data="np_mynp-2/data.yaml", epochs=100, imgsz=640)

Ultralytics YOLOv8.2.72 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/ultralytics/ultralytics/cfg/models/v8/backbonev9.yaml, data=np_mynp-2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

100%|██████████| 755k/755k [00:00<00:00, 14.9MB/s]
2024-08-04 06:00:12,827	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-04 06:00:14,200	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       232  ultralytics.nn.modules.conv.Conv             [3, 8, 3, 2]                  
  1                  -1  1      1184  ultralytics.nn.modules.conv.Conv             [8, 16, 3, 2]                 
  2                  -1  1      1888  ultralytics.nn.modules.block.C2f             [16, 16, 1, True]             
  3                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  4                  -1  2     12544  ultralytics.nn.modules.block.C2f             [32, 32, 2, True]             
  5                  -1  1      2880  ultralytics.nn.modules.block.SCDown          [32, 64, 3, 2]                
  6                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  7                  -1  1      9

100%|██████████| 6.25M/6.25M [00:00<00:00, 75.0MB/s]


AMP: checks passed ✅


...

    100/100      1.81G     0.7268     0.4825      1.136         22        640: 100%|██████████| 193/193 [00:26<00:00,  7.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  5.89it/s]

                   all        268        347      0.897      0.818      0.904      0.623



100 epochs completed in 0.876 hours.
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train2/weights/last.pt, 4.0MB
Optimizer stripped from /kaggle/working/ultralytics/runs/detect/train2/weights/best.pt, 4.0MB

Validating /kaggle/working/ultralytics/runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.72 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
backbonev9 summary (fused): 184 layers, 1,865,174 parameters, 0 gradients, 5.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:03<00:00,  2.69it/s]


                   all        268        347      0.893      0.825      0.907      0.626
                   FIT        125        162      0.888       0.79      0.894      0.665
                 UNFIT        163        185      0.898      0.859      0.919      0.587
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /kaggle/working/ultralytics/runs/detect/train2


In [ ]:
model.val(data="np_mynp-2/data.yaml")

Ultralytics YOLOv8.2.72 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
backbonev9 summary (fused): 184 layers, 1,865,174 parameters, 0 gradients, 5.5 GFLOPs


val: Scanning /kaggle/working/np_mynp-2/test/labels.cache... 268 images, 1 backgrounds, 0 corrupt: 100%|██████████| 268/268 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  4.42it/s]


                   all        268        347      0.893      0.825      0.907      0.626
                   FIT        125        162      0.888       0.79      0.894      0.665
                 UNFIT        163        185      0.898       0.86      0.919      0.587
Speed: 0.2ms preprocess, 3.6ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to /kaggle/working/ultralytics/runs/detect/train22


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7e7ecde5f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
!zip -r out.zip /kaggle/working/ultralytics/runs/detect

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


  adding: kaggle/working/ultralytics/runs/detect/ (stored 0%)
  adding: kaggle/working/ultralytics/runs/detect/train22/ (stored 0%)
  adding: kaggle/working/ultralytics/runs/detect/train22/F1_curve.png (deflated 12%)
  adding: kaggle/working/ultralytics/runs/detect/train22/val_batch2_labels.jpg (deflated 5%)
  adding: kaggle/working/ultralytics/runs/detect/train22/val_batch1_pred.jpg (deflated 5%)
  adding: kaggle/working/ultralytics/runs/detect/train22/val_batch1_labels.jpg (deflated 5%)
  adding: kaggle/working/ultralytics/runs/detect/train22/val_batch0_labels.jpg (deflated 4%)
  adding: kaggle/working/ultralytics/runs/detect/train22/val_batch0_pred.jpg (deflated 4%)
  adding: kaggle/working/ultralytics/runs/detect/train22/val_batch2_pred.jpg (deflated 4%)
  adding: kaggle/working/ultralytics/runs/detect/train22/confusion_matrix_normalized.png (deflated 35%)
  adding: kaggle/working/ultralytics/runs/detect/train22/P_curve.png (deflated 14%)
  adding: kaggle/working/ultralytics/runs/d

In [ ]:
%ls

np_mynp-2/  out.zip  ultralytics/  yolov8n.pt


In [ ]:
from IPython.display import FileLink
FileLink(r'out.zip')

/kaggle/working/out.zip

In [ ]:
ls

np_mynp-2/  out.zip  ultralytics/  yolov8n.pt
